In [1]:
# %%bash  
# pip install pillow #Requirement already satisfied: pillow in /root/opt/anaconda3/lib/python3.5/site-packages
# apt-get install libsm6 libxrender1 libfontconfig1

# pip install matplotlib
# gsutil cp gs://accelerometer-d41e2.appspot.com/accelerometer-d41e2-firebase-adminsdk-h782h-b747ff5f64.json /home/sshfile.json
# gsutil cp gs://dirversdatatest/Testing-dc8f6bcff8e2.json /home/sshfiletest.json
# gsutil cp gs://visuallab/mall_dataset/perspective_roi.mat /home/vis/perspective_roi.mat 
# gsutil cp gs://visuallab/mall_dataset/mall_gt.mat /home/vis/mall_gt.mat 
# gsutil cp gs://visuallab/mall_dataset/mall_feat.mat /home/vis/mall_feat.mat 
# gsutil cp gs://visuallab/mall_dataset/frames/*.jpg /home/vis/

# gsutil cp gs://visuallab/mall_dataset/perspective_roi.mat /home/vis/perspective_roi.mat 
# gsutil cp gs://visuallab/mall_dataset/mall_gt.mat /home/vis/mall_gt.mat 
# gsutil cp gs://visuallab/mall_dataset/mall_feat.mat /home/mayakurtser-putty/matlab_mall/mall_feat.mat 
# conda update mkl

In [2]:
import matplotlib as plt
plt.use('agg')
%matplotlib inline

from PIL import Image
import scipy
import scipy.io
import scipy.stats as stats
import pickle

from matplotlib.pyplot import imshow
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import pylab as pltl
import matplotlib.image as mpimg
import numpy as np 
from scipy.stats import multivariate_normal
import pandas as pd
from scipy import ndimage
import random
import scipy.ndimage
import itertools

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib
tf.logging.set_verbosity(tf.logging.INFO)

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
#roi = scipy.io.loadmat('/home/vis/perspective_roi.mat')
#mall_gt = scipy.io.loadmat('/home/vis/mall_gt.mat')
#mall_feat = scipy.io.loadmat('/home/vis/mall_feat.mat')

## Replace the above paths with this: ##
# /home/vis/ => /home/mayakurtser-putty/matlab_mall/
# importing the data and convert it from a matlab object into a dictionary
roi = scipy.io.loadmat('/home/mayakurtser-putty/matlab_mall/perspective_roi.mat')
mall_gt = scipy.io.loadmat('/home/mayakurtser-putty/matlab_mall/mall_gt.mat')
mall_feat = scipy.io.loadmat('/home/mayakurtser-putty/matlab_mall/mall_feat.mat')

FileNotFoundError: [Errno 2] No such file or directory: '/home/mayakurtser-putty/matlab_mall/perspective_roi.mat'

In [11]:
# saving the data in more comfortable way
pictures = mall_gt['frame'][0]
pmap = roi['pMapN']

In [12]:
#iterates all the heads of an image and for all the heads calculate the gaussian 
# pdf in every pixel in the square that is in the length of 2*3.5*std which center
# is the head coordinates

def calcHeads(z,matrix,hight,width):
    
    mekadem = 5 #this mekadem been choosed ampiracly by trail and error in the article the mekadem was 0.2 but it was too small
    
    mekadem_body = 2.5 #this mekadem is alsow been choosed ampiracly by trail and error 
    
    l = 3.5 * mekadem * pmap[int(480-z[1])][int(z[0])] #this is the length of half the square from the center to the edge 
    #(3.5*sigma over this 
    #value the values tends to drop to 0)
   
    
#     mu_body = [z[0],max(z[1]-l-1.75 * pmap[int(480-(z[1]+l+1.75))][int(z[0])],0)]
    
#     k = 3.5 * mekadem_body * mekadem * pmap[int(480-mu_body[1])][int(mu_body[0])]
    
#     Z = multivariate_normal.pdf([z[1],z[0]],mean=[z[1],z[0]],
#                 cov=[[mekadem*pmap[int(480-z[1])][int(z[0])],0]
#             ,[0,mekadem*pmap[int(480-z[1])][int(z[0])]]]) 
    
#     Z = Z + multivariate_normal.pdf([mu_body[1],mu_body[0]],mean=[mu_body[1],mu_body[0]],
#                 cov=[[mekadem_body*mekadem*pmap[int(480-mu_body[1])][int(mu_body[0])],0]
#                      ,[0,mekadem*pmap[int(480-mu_body[1])][int(mu_body[0])]]])

    
    for i in range (max(int(z[1]-l),0),min(int(z[1]+l),hight)):
        for j in range (max(int(z[0]-l),0),min(int(z[0]+l),width)):
            #this inner loop is to calculate values to the square of the head described in variable "z" 
            matrix[i][j] = matrix[i][j] + multivariate_normal.pdf([i,j],mean=[z[1],z[0]],
                cov=[[mekadem*pmap[int(480-z[1])][int(z[0])],0]
                     ,[0,mekadem*pmap[int(480-z[1])][int(z[0])]]]) 
#             / Z
            
#     for i in range (max(int(mu_body[1]-k),0),min(int(mu_body[1]+k),hight)):
#         for j in range (max(int(mu_body[0]-l),0),min(int(mu_body[0]+l),width)):
#             matrix[i][j] = matrix[i][j] + multivariate_normal.pdf([i,j],mean=[mu_body[1],mu_body[0]],
#                 cov=[[mekadem_body*mekadem*pmap[int(480-mu_body[1])][int(mu_body[0])],0]
#                      ,[0,mekadem*pmap[int(480-mu_body[1])][int(mu_body[0])]]]) / Z
   

In [13]:
# this function is calculating the density map for a speciic picture
def dense_map(width,hight,pic):
    print('pic:')
    print(pic)
    matrix = np.zeros((hight+1,width+1))#initialize the matrix with zero values
    heads = pictures[pic][0][0][0]#save the heads in the picture
    list(map(lambda z:calcHeads(z,matrix,hight,width),heads))#iterate all heads in pic and calculate there square pixel values
    return matrix

In [ ]:
# this box is creating the PDF for all the pictures
width = 639
hight = 479
# num_of_pictures = 2000 #this is the real line needs to be print because of the picture number
num_of_pictures = 2000 #this runs for testing only
print('den_map started')
den_map = [dense_map(width,hight,pic) for pic in range (800,num_of_pictures)]#create a list of length num_of_pictures 
#of density maps
print('den_map finished')

den_map started
pic:
800


In [ ]:
def density_patch(x0 , y0 , x1 , y1 , pic):
    #crops the patch with cordinates x0,x1,y0,y1 from the density map of pic    
    
    back = [] # the patch to return
    dense = den_map[pic] #saving the pic density map from the density maps list
    counter = 0 #counting the rows ti know in which row to shove the columns
    for i in range(y0,y1): #loopin over the delta y
        
        back.append([]) #new row
        col = []
        for j in range(x0,x1):#looping over the delta x
            col.append(den_map[pic][hight-i][width-j]) #adding new pixel from the column
        back[counter] = col #adding the entire col to the row
        counter+=1 # next row
        
    return back

In [ ]:
def get_dense(x0 , y0 , x1 , y1 , pic):
    dim = 18 #the length of the patch after resize
    dense = density_patch(x0 , y0 , x1 , y1 , pic)#getting the dense patch
    back1 = scipy.ndimage.zoom(dense, dim/len(dense), order=1)#resizing the density patch to 18X18
    return back1

In [ ]:
def get_count(x0 , y0 , x1 , y1 , pic):
    
    back = 0 #heads counter
    heads = pictures[pic][0][0][0] #gettint the list of heads in pic
    
    for i in heads: #itterate over all heads and add 1 to counter if the head is in delta x delta y i.e in the patch
        if x0 <= width - i[0] and x1>=width - i[0] and y0 <=hight- i[1] and y1>=hight - i[1]:
            back+= 1
            
    return back

In [ ]:
def get_count2(x0 , y0 , x1 , y1 , pic):
    #countint the number of heads in the picture by integraling over the patch from the dense map
    dense = density_patch(x0 , y0 , x1 , y1 , pic)     
    return sum(sum(dense,[]))

In [ ]:
patches_per_pic = 80 #number of patches
patches = [] #the patches from all the pictures (picture(0-1999),number of patch(0-79))
patches_rgb = [] #the patches from all the pictures with the rgb value in each pixel (picture(0-1999),number of patch(0-79))
tuples = [] # the delta x and delta y of each patches (x0,y0,x1,y1) (picture(0-1999),number of patch(0-79))
densities = [] # the patches from all the density maps (picture(0-1999),number of patch(0-79))
counts = [] # the counts of people in each patch by iterate heads (picture(0-1999),number of patch(0-79))
counts2 = [] # the counts of people in each patch by integration (picture(0-1999),number of patch(0-79))
examples = [] # the patches that has people in it (picture(0-1999),number of patch(0-79))

for i in range(0,num_of_pictures):#iterate over all the pictures
  
    #add a new list to all the lists, all the patches to pic i
    patches.append([]) 
    patches_rgb.append([])
    tuples.append([])
    densities.append([])
    counts.append([])
    counts2.append([])
    #the path of the picture
    im_path = '/home/mayakurtser-putty/matlab_mall/frame/seq_' + str(i+1).rjust(6,'0') + '.jpg'

    #import the pic and rotate it because it's upside-down
    img = Image.open(im_path,"r").rotate(180)
    
    for j in range(0,patches_per_pic): #iterate over all the patches
        
        flag = True #indicates that the delta x, delta y are ligable
        
        while flag:
            
            y = random.randint(0,hight) #randomly choosing the center of a square - y coordinate
            x = random.randint(0,width) #randomly choosing the center of a square - x coordinate
            ranges = int(pmap[int(479-y)][x])  *  20 # the length of the square 
            l = int(ranges/2) #l is the half the length of the square the distance between the center of the square to the edge
            
            if x - l >= 0 and x + l < width and y - l >= 0 and y + l < hight and l!=0: #checking if the length is leagiable

                tup = (x - l,y - l ,x + l,y + l) #the delta x delta y
                patch = img.crop(tup).resize((72, 72), Image.ANTIALIAS) #crop the patch
                patches[i].append(patch) #add to the list
                patches_rgb[i].append(list(patch.getdata())) #change to rgb
                tuples[i].append(tup) #add to tuples list
                densities[i].append(get_dense(x-l,y-l,x+l,y+l,i)) #croping the patch fron the density map
                counts[i].append(get_count(x-l,y-l,x+l,y+l,i)) #counting by heads list
                counts2[i].append(get_count2(x-l,y-l,x+l,y+l,i)) #counting by integral
                if counts[i][j] > 1: #saving the patches with people in it
                    aaa = i
                    bbb = j
                    examples.append([i,j])
                flag = False # the crop is legiable 
                

In [ ]:
#just for testing
aaa = examples[0][0]
bbb = examples[0][1]

In [ ]:
#prints for tests
print(aaa,bbb)
print(patches[aaa][bbb])#checked is good
print(tuples[aaa][bbb])#checked represent the right messure
print(len(densities[aaa][bbb]))
print(counts[aaa][bbb])

In [ ]:
testing an example patch
im = imshow(patches[aaa][bbb],origin="Upper")

In [ ]:
#testing the patch of the density map
side = np.linspace(0,len(densities[aaa][bbb]),len(densities[aaa][bbb]))
side1 = np.linspace(0,len(densities[aaa][bbb]),len(densities[aaa][bbb]))
X,Y = np.meshgrid(side,side1)
a = np.matrix(densities[aaa][bbb])
dt = pd.DataFrame(data=a)

dense2 = imshow(X=dt,interpolation='bilinear', cmap=cm.cubehelix,
                origin='Upper',
                vmax=abs(a).max())

In [ ]:
#testing to see a picture
im = imshow(Image.open('/home/mayakurtser-putty/matlab_mall/frame/seq_000001.jpg').rotate(180),origin='Upper')

In [ ]:
#testing to see if a density map is legiable
side = np.linspace(0,479,480)
side1 = np.linspace(0,639,640)
X,Y = np.meshgrid(side,side1)
a = np.matrix(den_map[0])
dt = pd.DataFrame(data=a)

dense2 = imshow(X=ndimage.rotate(dt,180),interpolation='bilinear', cmap=cm.cubehelix,
                origin='Upper',
                vmax=abs(a).max())

In [ ]:
#testing to see the density on top of the picture
im = imshow(Image.open('/home/mayakurtser-putty/matlab_mall/frame/seq_000001.jpg').rotate(180))

dense = imshow(X=ndimage.rotate(dt,180),interpolation='bilinear'
               , 
               cmap=cm.cubehelix,
                origin='Lower',
                vmax=abs(a).max(),alpha=0.5)

In [ ]:
#flatning the input data to a list of patches - length = (2000*80)
patches_rgb = [j for i in patches_rgb for j in i]#features
densities = [j for i in densities for j in i]#labels 1
counts = [j for i in counts for j in i]# labels 2

In [ ]:
#testing only
print(len(patches_rgb[0][0]))
print(len(densities[0]))
print(len(counts))

In [ ]:
#testing only
len(densities[0])

In [ ]:
# Load training and eval data
train_length = int(0.7*len(patches_rgb))

train_data = np.asarray(patches_rgb[0:train_length],dtype=np.float32)
train_labels1 = np.asarray(densities[0:train_length],dtype=np.float32)
train_labels2 = np.asarray(counts[0:train_length],dtype=np.float32)

eval_data = np.asarray(patches_rgb[train_length:],dtype=np.float32)
eval_labels = np.asarray(densities[train_length:],dtype=np.float32)


In [ ]:
#testing only
print(train_data.shape)
print(train_labels1.shape)
print("**for testing, printing one density map**")
print(train_labels1[100])

In [ ]:
pickle.dump( den_map, open( "den_map.p", "wb" ) )
pickle.dump( patches, open( "patches.p", "wb" ) )
pickle.dump( patches_rgb, open( "patches_rgb.p", "wb" ) )
pickle.dump( tuples, open( "tuples.p", "wb" ) )
pickle.dump( densities, open( "densities.p", "wb" ) )
pickle.dump( counts, open( "counts.p", "wb" ) )
pickle.dump( counts1, open( "counts1.p", "wb" ) )
pickle.dump( examples, open( "examples.p", "wb" ) )
pickle.dump( train_data, open( "train_data.p", "wb" ) )
pickle.dump( train_labels1, open( "train_labels1.p", "wb" ) )
pickle.dump( train_labels2, open( "train_labels2.p", "wb" ) )
pickle.dump( eval_data, open( "eval_data.p", "wb" ) )
pickle.dump( eval_labels, open( "eval_labels.p", "wb" ) )